In [1]:
import pandas as pd
import numpy as np
from rfm import RFM
from sklearn.model_selection import train_test_split
import scipy.integrate
import scipy.special

Загружаем файлы с транзакциями и клиентами

In [2]:
df_clients = pd.read_csv("./clients_small.csv", sep=';')
df_transactions = pd.read_csv("./transactions_small.csv", sep=';')


Общий план: 
1. Проводим RFM - сегментацию
2. Приводим в порядок возраст клиентов 
3. Приводим в порядок пол клиентов
4. Смотрим корреляцию и выделяем параметры которые сильнее всего коррелируют с частотой 
5. Пробуем разные разбиение по бинам и смотрим средне по дисперсии по всем бинам при разым разбиениям 
6. Для лучшего разбиения по бинам запускаем стратификацию и смотрит на Kolmogorov–Smirnov test

In [3]:
df_clients.head(3)

,customer_id,card_id,region_name,sex,birth_date
0,02F4DB34C5696A668068CA3C1DE7C2D5,6600B4FDA036A6F00D464541EAD3B21F,Москва,F,1938-08-18
1,0308713D02D676920F2961E2A84B4516,1FB9344B8661069910D184EF19C448C7,Сахалинская область,F,1990-03-26
2,032672FDC2CF85213079F83E447AF75B,4C8AD60064AF412753EB54EAFCD8C59D,Самарская область,F,1989-01-13


In [4]:
df_transactions.head(3)

,customer_id,card_id,tr_datetime,tr_normalized_amount,currency,mcc,merchant_name,merchant_city
0,634E7564849D27B3C452229E405629A6,52CD1B87531555F5611C663562FC686D,2019-09-07,10.00,810,5999,i-bank ┴шыaщэ >moscow ru,moscow
1,B20A638E7EB926C63102144596F38C71,9D4F087597B65028953FBE8A8FBF48DE,2019-09-20,313.04,810,5411,dixy moskva ru,moscow
2,125EBE0A38873D4888408F85877282EF,B8859E8DB421C73B785521A6252A36CB,2019-09-23,250.00,810,5999,i-bank teыe 2 >moscow ru,moscow


1. Проводим RFM - сегментацию

In [5]:
r = RFM(df_transactions, customer_id='customer_id', transaction_date='tr_datetime', amount='tr_normalized_amount')

In [6]:
df_transactions['tr_datetime'] = pd.to_datetime(df_transactions['tr_datetime'])

In [7]:
df_clients_rfm = r.rfm_table

In [8]:
df_clients_rfm[df_clients_rfm['recency'] == 90].shape[0]

1

In [9]:
df_clients_rfm.pivot_table(columns=['r'],index=['f'], values = ['customer_id'], aggfunc= 'count',fill_value=0).reset_index()

f customer_id                    
r              1    2    3    4    5
0  1         542  220  126  100   86
1  2         242  280  200  179  173
2  3         152  255  231  199  237
3  4          89  204  247  261  273
4  5          49  115  270  335  305

In [10]:
df_clients = df_clients[['customer_id','region_name','sex','birth_date']].merge(df_clients_rfm[['r','f','m','frequency']], left_index=True, right_index=True)


2. Приводим в порядок возраст клиентов 

In [11]:
#Меняем тип даты рождения в датафрейме  
df_clients['birth_date'] = pd.to_datetime(df_clients['birth_date'], errors='coerce')

In [12]:
#Разбиываем даты рождения на бины 
df_clients['birth_date_bin'] = pd.cut(df_clients['birth_date'], bins=5, labels=False) + 1

In [13]:
#Заполяем бины для тех кто у кого нет даты рождения нулями 
df_clients.loc[df_clients['birth_date_bin'].isnull(),'birth_date_bin'] = 0

In [14]:
#Приводим значения бинов к типу int 
df_clients['birth_date_bin'] = df_clients['birth_date_bin'].astype(int)

3. Приводим в порядок пол клиентов

In [15]:
#Ищем есть ли клиенты с незаполненным полом 
df_clients[df_clients['sex'].isna()].shape[0]

131

In [16]:
#Заполняем нулями незаполненный пол 
df_clients['sex'] = df_clients['sex'].fillna(0)

In [17]:
#Заполняем -1 и 1 женский и мужской пол
df_clients['sex'] = df_clients['sex'].replace({ 'F' : -1, 'M' : 1})

4. Смотрим корреляцию и выделяем параметры которые сильнее всего коррелируют с частотой 

In [18]:
df_clients.corr()

,sex,r,f,m,frequency,birth_date_bin
sex,1.000000,-0.024664,-0.001333,-0.010932,-0.001138,0.005436
r,-0.024664,1.000000,0.390317,0.289479,0.364000,-0.012040
f,-0.001333,0.390317,1.000000,0.668715,0.866190,-0.023198
m,-0.010932,0.289479,0.668715,1.000000,0.629928,-0.030099
frequency,-0.001138,0.364000,0.866190,0.629928,1.000000,-0.033979
birth_date_bin,0.005436,-0.012040,-0.023198,-0.030099,-0.033979,1.000000


Смотрим, какие переменные больше всего коррелируют с frequency, это:
1. f
2. m
3. r
От sex и birth_date_bin зависимость небольшая. \
---------------------------------------------------

5. Пробуем разные разбиение по бинам
- Для каждого бина в разбиении считаем дисперсию в качестве переменной
- Считаем среднее между дисперсиями по всем бинам 
- Чем ниже будет среднее тем более качественное разделение   


In [19]:
df_clients.pivot_table(columns=['r'],index=['f'],values = ['frequency'],aggfunc= 'std',fill_value=0).reset_index().mean().mean()
# по r и f значение 20.57

20.569667413126272

In [20]:
df_clients.pivot_table(columns=['m'],index=['f'],values = ['frequency'],aggfunc= 'std',fill_value=0).reset_index().mean().mean()
# по m и f значение 15.49

15.498378273719192

In [21]:
df_clients.pivot_table(index=['region_name'],values = ['frequency'],aggfunc= 'std',fill_value=0).reset_index().mean().mean()
# по region_name значение 79.92 
# Деление по регионам плохая идея 

79.92672723733817

Судя по результатам лучшее для стратефикации разбиение по бинам это с использованием m и f \
[-------------------------------------------------------------------------------

6. Для лучшего разбиения по бинам запускаем стратификацию и смотрит на Kolmogorov–Smirnov test

In [22]:
#производим разделение на группы A и B с учетом стратификации по бинам
A_group, B_group= train_test_split(df_clients, test_size=0.5, stratify = df_clients[['m','f']])

In [23]:
#смотрим результаты теста Колмогорова-Смирнова
scipy.stats.ks_2samp(A_group['frequency'], B_group['frequency'], alternative='two-sided', mode='auto')
#statistic=0.0100 - хороший результат 

KstestResult(statistic=0.01303538175046555, pvalue=0.976529905515252)